In [ ]:
import os
from kaggle_datasets import KaggleDatasets
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# tensorflow stuff
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

# visualization
import matplotlib.pyplot as plt
plt.style.use('ggplot')

AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    tpu_strategy = tf.distribute.TPUStrategy(tpu)
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
except ValueError:
    strategy = tf.distribute.MirroredStrategy()
    
print("Number of accelerators: ", tpu_strategy.num_replicas_in_sync)

In [ ]:
GCS_PATH = KaggleDatasets().get_gcs_path()
GCS_PATH

In [ ]:
!gsutil ls $GCS_PATH

In [ ]:
# Main dir on GCS
main_dir = os.path.join(GCS_PATH, "tfrecords-jpeg-224x224/")

# creating paths dictionary
paths = dict()
paths['train'], paths['val'], paths['test'] = None, None, None

for k, v in paths.items():
    paths[k] = tf.io.gfile.glob(main_dir + k + '/*.tfrec')
# paths

# **Constants**

In [ ]:
IMAGE_SHAPE = [224, 224, 3]
BATCH_SIZE = 16 * tpu_strategy.num_replicas_in_sync
NUM_CLASSES = 104
TRAIN_SIZE = 12753
VALIDATION_SIZE = 16 * 232
TRAIN_STEPS = TRAIN_SIZE // BATCH_SIZE
VALIDATION_STEPS = VALIDATION_SIZE // BATCH_SIZE

# **Data Retrieval**

In [ ]:
def augment_image(image, label):
    aug_image = tf.image.random_brightness(image, 0.6)
    aug_image = tf.image.random_flip_left_right(aug_image)
    aug_image = tf.image.random_flip_up_down(aug_image)
    return aug_image, label

def load_training_data(apply_augmentation=True):
    dataset = load_dataset(paths['train'], labeled=True)
    if apply_augmentation:
        dataset = dataset = dataset.map(augment_image, num_parallel_calls=AUTO)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

def load_validation_data():
    dataset = load_dataset(paths['val'], labeled=True)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

def load_test_data():
    dataset = load_dataset(paths['test'], labeled=False)
    dataset = dataset.batch(BATCH_SIZE)
    
    return dataset

def parse_image(image_bytes):
    image = tf.image.decode_jpeg(image_bytes, channels=3)
    image = tf.reshape(image, IMAGE_SHAPE)
    
    return image

def load_dataset(paths, labeled):
    # read from files
    dataset = tf.data.TFRecordDataset(paths, num_parallel_reads=AUTO)
    
    dataset = dataset.with_options(ignore_order)
    # map tfrecord to normal data
    dataset = dataset.map(
        parse_labeled_data if labeled else parse_unlabeled_data)
    
    return dataset

    
def parse_labeled_data(example):
    IMAGE_MESSAGE = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'class': tf.io.FixedLenFeature([], tf.int64),
    }
    image_data = tf.io.parse_single_example(example, IMAGE_MESSAGE) 
    image = parse_image(image_data['image'])
    label = tf.cast(image_data['class'], tf.int32)
    
    return image, label

def parse_unlabeled_data(example):
    IMAGE_MESSAGE = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'id': tf.io.FixedLenFeature([], tf.int64), 
    }
    image_data = tf.io.parse_single_example(example, IMAGE_MESSAGE)
    image_id = tf.cast(image_data['id'], tf.int32)
    
    return image, image_id
    
    

In [ ]:
# for (x,y) in load_training_data().take(1):
#     plt.imshow(x[0])
#     plt.imshow(tf.image.random_brightness(x[0], 0.6))
#     break

In [ ]:
print(load_training_data())

In [ ]:
# data dump
print("Training data shapes:")
for image, label in load_training_data().take(3):
    print(image.numpy().shape, label.numpy().shape)
print("Training data label examples:", label.numpy()[:15])

In [ ]:
# labels, counts = np.unique(train_y, return_counts=True)
# max_count = max(counts)
# normalized_counts = counts / max_count
# my_cmap = plt.get_cmap('viridis_r')
# colors = my_cmap(normalized_counts)
# fig, ax = plt.subplots(figsize=(20,30))
# count_graph = ax.barh(labels, counts, color=colors)
# ax.set_title('Class Count Histogram')
# ax.set_yticks(labels);

In [ ]:

# img = train_x[0]
# plt.axis('off')
# plt.imshow(img)
# print(f'Label : {train_y[0]}')
# print(f'Image shape : {img.shape}')

In [ ]:

with tpu_strategy.scope():
    # defining the layers
    dense = layers.Dense(NUM_CLASSES, activation='softmax', name='Last-Layer')
    core = keras.applications.EfficientNetB6(weights='imagenet', include_top=False, pooling='avg')
    preproc_img = layers.Lambda(lambda data: 
                                keras.applications.efficientnet.
                                    preprocess_input(tf.cast(data, tf.float32)), input_shape=IMAGE_SHAPE)
    # model process
    i = layers.Input(shape=IMAGE_SHAPE)
    x = preproc_img(i)
    x = core(x)
    x = dense(x)

    # creating model
    model = keras.Model(inputs=[i], outputs=[x])
    
    # create an optimizer
    optimizer = keras.optimizers.Adam()
    # create a loss function
    loss_fn = keras.losses.SparseCategoricalCrossentropy()
    # create metrics
    metrics = [keras.metrics.SparseCategoricalAccuracy()]

    model.compile(
        optimizer=optimizer,
        loss =loss_fn,
        metrics=metrics,
        steps_per_execution=8
        )




In [ ]:
model.summary()

In [ ]:
history = model.fit(
    load_training_data(),
    validation_data=load_validation_data(),
    epochs=30,
    steps_per_epoch=TRAIN_STEPS,
    validation_steps=VALIDATION_STEPS
    )

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.legend(['train loss', 'val loss'], loc='upper right');

In [ ]:
plt.plot(history.history["sparse_categorical_accuracy"])
plt.plot(history.history["val_sparse_categorical_accuracy"])
plt.legend(['train acc', 'val acc'], loc='upper left');